In [45]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, RepeatVector
from keras.layers import LSTM
from keras.layers import Conv1D
from keras.layers import MaxPooling1D

from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


In [46]:
path="C:\\Users\\nived\\Desktop\\Nivi ideapad Lenovo\\Nivoo SJ\\SJSU sem 2\\CS256\\project\\CS256_Backup\\irish-music2.txt"
with io.open(path, encoding='latin-1') as f:
    text = f.read()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []

#sentences=text.split("\n")
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


corpus length: 8591
total chars: 68
nb sequences: 2851
Vectorization...


In [47]:
def build_model(maxlen, chars):
    # build the model: an MLP with 1 hidden layer
    print('Build model...')
    model = Sequential()
    model.add(LSTM(100, input_shape=(maxlen, len(chars))))  # encoder lstm
    model.add(RepeatVector(28))
    model.add(LSTM(50, return_sequences=True))  # decoder lstm
    model.add(Flatten())
    model.add(Dense(len(chars), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    print(model.summary())
    return model



In [48]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [49]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [50]:
if __name__ == '__main__':
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model = build_model(maxlen, chars)

    model.fit(
        x,
        y,
        batch_size=128,
        epochs=5,
        callbacks=[print_callback]
    )

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 100)               67600     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 28, 100)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 28, 50)            30200     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1400)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 68)                95268     
Total params: 193,068
Trainable params: 193,068
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
2851/2851 [==============================] - 5s 2ms/step - loss: 3.3340 - acc: 0.1459

----- Generating te

2 (dcAB) | G2G2 G3G | B2d2 (gfdc) | d2g2 2 22 2 22 2 2 2-- 2--- -d-d--- d-d d-2-- c2--- d--d--d-d-d d2- d--d- dd2 2-- d2 2-- 2---- d-d--d d-d--2 d--d-d d2----d- d--d- d2----d-- -d--------d d-d---d- d-cd-2 2 -d--d d2- d2-- 2----- d2--d- d--d---d d2---- d-d-2 -----d--d--------
---d- d---d d-d-2- 3--- d----d--3- c--d- d--d---d--- dd--d 2---- d2--- -d---d- d2--d 2-------d-d-d- d-d2- d- d22 --d d2 d2 2- 2---- -d----d d--cd-2- --d2- -d-d- d-d
----- diversity: 0.5
----- Generating with seed: "2 (dcAB) | G2G2 G3G | B2d2 (gfdc) | d2g2"
2 (dcAB) | G2G2 G3G | B2d2 (gfdc) | d2g2 |d dd22 2 2-d |!
dd d2 dd|cB 2 AA22 22:|!


d cAc|AB BA B|GF F AF|F BA 2222::|
!

 BdA/ce//d|dB BA A22|!
!
f/d Bd|dBA BA| Bdd|B BAA BAB|2 B BA|A A22 B22|!
!

A B2|A dB|BA BA AB|F BA F/|A BA|e dcd22|!
 d22 ec| Bdd B2|A c|B2 B Ae|d B22 :|!

d BA2|B AAA B2| BBA AA22|!

d B BcA|B B2 c|!
A AcA|d BB AE|A BA|A AA22:|!

d e| dB/d/e Ac|A BF E2|!

A A2|B BA/ dA22:|!

dA BA BA|B B2 AA|FA BA F|A! GA AB/| BA 
----- diversity: 1.0
-----